<a id=contents></a>

# Cleaning notebook
## Subtitle


[1. Data Inspection](#insp)

[2. Cleaning numerical data](#numerical)

[3. Cleaning categorical data](#categ)

[4. Cleaning text data](#text)

In [1]:
%load_ext autoreload
%autoreload 2 

import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as shc

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style("darkgrid")

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from sklearn.linear_model import LogisticRegression
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from matplotlib import cm
import numpy as np
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import string

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
tokenizer = RegexpTokenizer(r'\b\w{3,}\b')
stop_words = list(set(stopwords.words("english")))
stop_words += list(string.punctuation)

import warnings
warnings.filterwarnings('ignore')
from pathlib import Path



<a id=insp ><a/> 

## 1. Data Inspection
    
[LINK to table of contents](#contents)

In [2]:
import functions.functions as fn

In [3]:
# 4-5 min to run
raw_path = Path('data/raw')
clean, cvt = fn.clean_tweet_text(raw_path)

In [13]:
clean.head()

,Unnamed: 0,tweet_id,datetime,display_name,tweet_text,User_id,extracted_twitter_handles,extracted_URLs,extracted_hashtags,clean_tweet_text,...,ukraine,want,way,week,well,work,world,would,year,yes
0,0,1579299680491823104,2022-10-10 02:36:00+00:00,ElonMusk,@CathieDWood 💯,44196397.0,[@CathieDWood],[],[],💯,...,0,0,0,0,0,0,0,0,0,0
1,1,1579263949945835521,2022-10-10 00:14:01+00:00,ElonMusk,@WholeMarsBlog It will,44196397.0,[@WholeMarsBlog],[],[],It will,...,0,0,0,0,0,0,0,0,0,0
2,2,1579262341904203777,2022-10-10 00:07:38+00:00,ElonMusk,@chicago_glenn @neiltyson Strange @Twitter,44196397.0,"[@chicago_glenn, @neiltyson, @Twitter]",[],[],Strange,...,0,0,0,0,0,0,0,0,0,0
3,3,1579258883977416705,2022-10-09 23:53:53+00:00,ElonMusk,@WholeMarsBlog I have no desire to become invo...,44196397.0,[@WholeMarsBlog],[],[],"I have no desire to become involved in wars, ...",...,0,0,0,0,0,0,0,0,0,0
4,4,1579235577492574209,2022-10-09 22:21:17+00:00,ElonMusk,@Teslarati Strong candidate to win most counte...,44196397.0,[@Teslarati],[],[],Strong candidate to win most counter-intuitiv...,...,0,0,0,0,0,0,0,0,1,0


In [17]:
clean.to_csv('data/clean/clean_data.csv')

In [22]:
# now selecting those parts of the data that will be used only classification, i.e. the vectorized columns
cols  =list(clean.columns)
start_cvt_feats = cols.index('clean_tweet_text')+1
cvt_cols  = clean.columns[start_cvt_feats:]
cvt_cols.insert(0, 'tweet_id')
feats_df = clean[cvt_cols]
feats_df.to_csv('data/clean/features/count_vect_feats.csv')

In [23]:
clean_display = clean[1: cols[:start_cvt_feats]]
clean_display.to_csv('data/clean/clean_display_data.csv')

,Unnamed: 0,tweet_id,datetime,display_name,tweet_text,User_id,extracted_twitter_handles,extracted_URLs,extracted_hashtags,clean_tweet_text
0,0,1579299680491823104,2022-10-10 02:36:00+00:00,ElonMusk,@CathieDWood 💯,4.419640e+07,[@CathieDWood],[],[],💯
1,1,1579263949945835521,2022-10-10 00:14:01+00:00,ElonMusk,@WholeMarsBlog It will,4.419640e+07,[@WholeMarsBlog],[],[],It will
2,2,1579262341904203777,2022-10-10 00:07:38+00:00,ElonMusk,@chicago_glenn @neiltyson Strange @Twitter,4.419640e+07,"[@chicago_glenn, @neiltyson, @Twitter]",[],[],Strange
3,3,1579258883977416705,2022-10-09 23:53:53+00:00,ElonMusk,@WholeMarsBlog I have no desire to become invo...,4.419640e+07,[@WholeMarsBlog],[],[],"I have no desire to become involved in wars, ..."
4,4,1579235577492574209,2022-10-09 22:21:17+00:00,ElonMusk,@Teslarati Strong candidate to win most counte...,4.419640e+07,[@Teslarati],[],[],Strong candidate to win most counter-intuitiv...
...,...,...,...,...,...,...,...,...,...,...
517711,115520,1558734519506767873,2022-08-14 08:37:23+00:00,MoEnergy_Saudi,@m99rr وعليكم السلام ورحمة الله وبركاته، تم ال...,4.148042e+09,[@m99rr],[],[],وعليكم السلام ورحمة الله وبركاته، تم التواصل ...
517712,115521,1558732969296236545,2022-08-14 08:31:14+00:00,MoEnergy_Saudi,@justonereal11 وعليكم السلام ورحمة الله وبركات...,4.148042e+09,[@justonereal11],[],[],وعليكم السلام ورحمة الله وبركاته، تم التواصل ...
517713,115522,1558049964693098496,2022-08-12 11:17:13+00:00,MoEnergy_Saudi,شبابنا... مصدر عز وفخر،\nبهم نبني حاضرنا ونواص...,4.148042e+09,[],[https://t.co/Ghv9dzOeIU],[],شبابنا... مصدر عز وفخر،\nبهم نبني حاضرنا ونواص...
517714,115523,1557738208493834241,2022-08-11 14:38:24+00:00,MoEnergy_Saudi,@_Tarfa22 وعليكم السلام ورحمة الله وبركاته، كر...,4.148042e+09,[@_Tarfa22],[],[],وعليكم السلام ورحمة الله وبركاته، كرماً التوا...


<a id=numerical ><a/> 

## 2. Cleaning numerical data
    
[LINK to table of contents](#contents)

<a id=categ ><a/> 

## 3. Cleaning categorical data
   
[LINK to table of contents](#contents)

<a id=text ><a/> 

## 4. Cleaning text data
    
[LINK to table of contents](#contents)